In [23]:
import os, sys, time
import rasterio

import networkx as nx
import geopandas as gpd
import pandas as pd

from shapely.geometry import MultiLineString

In [2]:
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append("../")
import GOSTnets as gn

In [3]:
# Define input data
pth = "./"
# Read in cleaned pickle from earlier analysis and convert to time
G = nx.read_gpickle(os.path.join(pth, 'tutorial_outputs', r'iceland_network_clean.pickle'))
G_time = gn.convert_network_to_time(G, distance_tag = 'length', road_col = 'infra_type', factor = 1000)
# Define origins and destinations
rek_grid_file = os.path.join(pth, "tutorial_data", "rek_grid.shp")
rek_pop_grid_file = rek_grid_file.replace(".shp", "_pop.shp")
churches_file = os.path.join(pth, "tutorial_data", "churches.shp")

In [4]:
rek_grid = gpd.read_file(rek_pop_grid_file)
in_churches = gpd.read_file(churches_file)
in_churches = in_churches.to_crs(rek_grid.crs)

In [5]:
# calculate the origins and destinations by snapping to the road network
origins_df = gn.pandana_snap_c(G_time, rek_grid, source_crs='epsg:4326', target_crs='epsg:4326',add_dist_to_node_col = True)
origins = list(set(origins_df['NN']))
destinations_df = gn.pandana_snap_c(G_time, in_churches, source_crs='epsg:4326', target_crs='epsg:4326',add_dist_to_node_col = True)
destinations = list(set(destinations_df['NN']))

In [9]:
nodes_gdf = gn.node_gdf_from_graph(G_time)
edges_gdf = gn.edge_gdf_from_graph(G_time)

In [10]:
obj_nodes = nx.shortest_path(G_time, source=origins[0], target=destinations[0], weight="time")

In [13]:
edges_gdf.columns

Index(['stnode', 'endnode', 'time', 'infra_type', 'id', 'key', 'mode', 'Type',
       'Wkt', 'osm_id', 'length', 'geometry'],
      dtype='object')

In [25]:
all_res = []
oIdx = 0
for org in origins:
    oIdx = oIdx + 1
    print(f'{oIdx} of {len(origins)}')
    for dest in destinations:
        obj_nodes = nx.shortest_path(G_time, source=org, target=dest, weight="time")
        all_edges = []
        for idx in range(0, len(obj_nodes) - 1):
            start_node = obj_nodes[idx]
            end_node = obj_nodes[idx + 1]
            cur_edge = edges_gdf.loc[(edges_gdf['stnode'] == start_node) & (edges_gdf['endnode'] == end_node), 'geometry'].iloc[0]
            all_edges.append(cur_edge)
        all_res.append([org, dest, MultiLineString(all_edges)])

3
10247
2057
8212
8216
4135
12337
2098
8241
8253
12352
14402
12365
4178
4185
14426
new_obj_2035
12386
6245
10365
12417
4230
4231
new_obj_1474
10378
4234
8338
12435
8343
12443
12444
2206
4257
12456
2223
10416
8379
200
203
8405
8409
8410
new_obj_2327
14557
6377
new_obj_2055
6384
12542
14591
12543
new_obj_2346
260
12551
14602
268
12557
2336
2339
294
4405
4408
14649
12604
10560
2369
321
4417
4416
334
339
8532
344
10597
2419
8564
381
4490
14732
6541
8599
4505
414
415
8606
new_obj_185
2471
423
12715
2479
14768
12723
4535
4542
10690
new_obj_2426
2510
6607
12752
4559
14803
new_obj_2081
476
8669
10721
10726
6630
4587
12797
10754
4616
8713
12828
6691
2598
14887
6697
2602
6699
new_obj_2199
2609
8761
12858
10827
4691
14942
4706
8809
12909
10868
4725
12918
8828
640
10880
642
6790
4743
10888
6795
14996
6819
12964
6820
12968
681
12971
691
696
10936
4808
6868
6872
2785
6907
764
15108
6919
2823
11015
782
6926
8979
11029
805
13100
2861
11065
6973
6975
15172
839
6989
848
13144
11100
4957
2912
15210
13168

In [26]:
all_results = pd.DataFrame(all_res, columns=['O','D','geometry'])
all_results.to_csv(os.path.join(pth, "tutorial_data","all_OD_links.csv"))
  

AttributeError: 'DataFrame' object has no attribute 'grouby'

In [28]:
all_results.columns

Index(['O', 'D', 'geometry'], dtype='object')

In [29]:
#Generate unique table counts
unq_res = pd.DataFrame(all_results.groupby(['O', 'D', 'geometry']).count())
unq_res.head()
                

TypeError: unhashable type: 'MultiLineString'

In [30]:
all_results.head()

,O,D,geometry
0,3,15195,"(LINESTRING (-21.7458194 64.0618473, -21.73667..."
1,3,15556,"(LINESTRING (-21.7458194 64.0618473, -21.73667..."
2,3,11173,"(LINESTRING (-21.7458194 64.0618473, -21.73667..."
3,3,3240,"(LINESTRING (-21.7458194 64.0618473, -21.73667..."
4,3,9673,"(LINESTRING (-21.7458194 64.0618473, -21.77722..."


In [31]:
len(all_res)

6135